In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

# Diretório base: a pasta onde está o notebook
BASE_DIR = Path().resolve()

# Subpastas organizadas
RAW = BASE_DIR / "data" / "raw"     # onde ficam os arquivos originais
OUT = BASE_DIR / "data" / "output"  # onde vão os resultados

print("Base dir :", BASE_DIR)
print("RAW dir  :", RAW)
print("OUT dir  :", OUT)


df = pd.read_csv(RAW/'heart_disease_dataset.csv', encoding='utf-8', sep=',')
print("Dimensões (linhas, colunas):", df.shape)

# 3) Inspecionar rapidamente
display(df.head(10))              # primeiras linhas
print("\nColunas:", list(df.columns))
print("\nTipos:")
print(df.dtypes)

print("\nTop 10 colunas com mais nulos:")
print(df.isna().sum().sort_values(ascending=False).head(10))